In [1]:
import sys
import os
import glob
import pandas as pd

import numpy as np
import sklearn
print (sklearn.__version__)

from sklearn import linear_model
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.externals import joblib
import seaborn as sns

from sklearn.model_selection import cross_val_score

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import cross_val_score
from sklearn.svm import  SVC

from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.externals.six import StringIO

from sklearn.ensemble import RandomForestClassifier,VotingClassifier,AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier

0.19.1


In [2]:
# ディレクトリ名
input_dir = ur"C:/Users/mirait/wk/git/compe_sales/input/"
output_dir = ur"C:/Users/mirait/wk/git/compe_sales/output/"

In [3]:
# 訓練データを読み込む
train_path = input_dir + ur"sales_train_v2.csv"
train_data = pd.read_csv(train_path, encoding="cp932",low_memory=False)
train_data.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [4]:
# テストデータを読み込む
test_path = input_dir + ur"test.csv"
test_data = pd.read_csv(test_path, encoding="cp932",low_memory=False)
test_data.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [5]:
# 訓練+テストを結合
merge_arr = [train_data, test_data]

In [6]:
# 商品データを読み込む
item_path = input_dir + ur"items.csv"
item_data = pd.read_csv(item_path, encoding="utf-8",low_memory=False)
item_data.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [7]:
# 商品カテゴリデータを読み込む
category_path = input_dir + ur"item_categories.csv"
category_data = pd.read_csv(category_path, encoding="utf-8",low_memory=False)
category_data.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [8]:
# 店舗データを読み込む
shop_path = input_dir + ur"shops.csv"
shop_data = pd.read_csv(shop_path, encoding="utf8",low_memory=False)
shop_data.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [9]:
# 分析に生かすために項目の特徴量を取得
def feature(df):
    
    df_feature = pd.DataFrame()
    
    for i in (range(len(df.columns))):
        tmp = pd.DataFrame()
        tmp = df.iloc[:, [i]]
        
        selList = list()
        record_cn = len(tmp) #レコード数
        column_name = tmp.columns[0] #カラム名
        value_type_cn = len(pd.value_counts(tmp.values.flatten())) #値の種類数(NAはカウント外)
        NA_cn = tmp.isnull().sum().values[0] #NA件数

        feature_list = list([record_cn, column_name, value_type_cn, NA_cn])
        now_column = pd.DataFrame(feature_list).T
        #print res1
        
        df_feature = pd.concat([df_feature, now_column], ignore_index=True)
    
    df_feature.columns = [u'レコード数', u'カラム名', 'v_count', u'NA件数']
        
    col_names = list(df_feature.columns)
    df_feature = df_feature.loc[:, col_names]

    return df_feature

In [10]:
# 訓練データの特徴量
feature_train = feature(train_data)
print len(feature_train)

6


In [11]:
# 「date」を年月日単位で項目を分離
train_data['year'] = train_data['date'].str.split('.',expand=True)[2]
train_data['month'] = train_data['date'].str.split('.',expand=True)[1]
train_data['day'] = train_data['date'].str.split('.',expand=True)[0]

In [12]:
# 年月日を数値型に変換
train_data['year'] = train_data['year'].astype(np.float32)
train_data['month'] = train_data['month'].astype(np.float32)
train_data['day'] = train_data['day'].astype(np.float32)

In [13]:
print train_data.head()

         date  date_block_num  shop_id  item_id  item_price  item_cnt_day  \
0  02.01.2013               0       59    22154      999.00           1.0   
1  03.01.2013               0       25     2552      899.00           1.0   
2  05.01.2013               0       25     2552      899.00          -1.0   
3  06.01.2013               0       25     2554     1709.05           1.0   
4  15.01.2013               0       25     2555     1099.00           1.0   

     year  month   day  
0  2013.0    1.0   2.0  
1  2013.0    1.0   3.0  
2  2013.0    1.0   5.0  
3  2013.0    1.0   6.0  
4  2013.0    1.0  15.0  


In [14]:
# 分離を確認できたため、「date」削除
train_data = train_data.drop(columns='date', axis=1)

In [15]:
print train_data.head()

   date_block_num  shop_id  item_id  item_price  item_cnt_day    year  month  \
0               0       59    22154      999.00           1.0  2013.0    1.0   
1               0       25     2552      899.00           1.0  2013.0    1.0   
2               0       25     2552      899.00          -1.0  2013.0    1.0   
3               0       25     2554     1709.05           1.0  2013.0    1.0   
4               0       25     2555     1099.00           1.0  2013.0    1.0   

    day  
0   2.0  
1   3.0  
2   5.0  
3   6.0  
4  15.0  


In [16]:
# 「item_id」をキーに「item_category_id」を取得
train_data_01 = train_data.merge(item_data, on = ['item_id'], how = "inner")

In [17]:
train_data_01 = train_data_01.drop(columns='item_name', axis=1)

MemoryError: 

In [18]:
print len(train_data_01)

2935849


In [19]:
# 日ごとの売り上げから月ごとの売り上げを算出
# 31日まで：1、3、5、7、8、10,12
# 30日まで：4、6、9、11
# 28日まで：2

thity_one_list = [1, 3, 5, 7, 8, 10, 12]
thity_list = [4, 6, 9, 11]
twenty_eight_list = [2]

# 結果を詰める
wk_list = list()

for i, r in train_data_01.iterrows():
    
    # 31日の月
    if (r['month'] in thity_one_list):
        wk_list.append(r['item_cnt_month'] * 31)
    # 30日の月
    elif (r['month'] in thity_list):
        wk_list.append(r['item_cnt_month'] * 30)
    # 2月
    else:
        wk_list.append(r['item_cnt_month'] * 28)
    
wk_df = pd.DataFrame(wk_list)
train_data_01[item_cnt_month] = wk_df    

MemoryError: 